In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache('cache.db'))
set_debug(True)

chat = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

chat.predict('how to make an italian carbonara')

# examples = [
#     {
#         "movie": "Pulp Fiction",
#         "answer": """
#         Here is the information about movie Pulp Fiction:
#           Director: Quentin Tarantino
#           Main Cast: John Travolta, Samuel L. Jackson, Uma Thurman
#           Budget: $8 million
#           Box Office Revenue: $213.9 million worldwide
#           Genres: Crime, Drama, Thriller
#           Simple Synopsis: Pulp Fiction is a 1994 American independent crime film that intertwines four tales of violence and redemption in Los Angeles, following two hitmen, a boxer, a gangster’s wife, and a pair of diner bandits, known for its non-linear narrative, sharp dialogue, and pop culture references.
#         """
#     },
#     {
#         "movie": "Eternal Sunshine of the Spotless Mind",
#         "answer": """
#         Here is the information about movie Eternal Sunshine of the Spotless Mind:
#           Director: Michel Gondry
#           Main Cast: Jim Carrey, Kate Winslet, Kirsten Dunst, Mark Ruffalo, Elijah Wood, Tom Wilkinson
#           Budget: $20 million
#           Box Office Revenue: $73.1 million worldwide
#           Genres: Drama, Romance, Sci-Fi
#           Simple Synopsis: A couple, Joel and Clementine, undergo a procedure to erase memories of each other after a painful breakup. As Joel relives their memories during the process, he fights to preserve their love, exploring themes of memory, identity, and the value of painful experiences in a non-linear, surreal narrative.
#         """
#     },
#     {
#         "movie": "There Will Be Blood",
#         "answer": """
#         Here is the information about movie There Will Be Blood:
#           Director: Paul Thomas Anderson
#           Main Cast: Daniel Day-Lewis, Paul Dano, Ciarán Hinds
#           Budget: $25 million
#           Box Office Revenue: $76.2 million worldwide
#           Genres: Drama, History
#           Simple Synopsis: Set in the early 20th-century American West, the film follows Daniel Plainview, a ruthless oil prospector whose relentless pursuit of wealth and power leads to greed, betrayal, and moral decay, clashing with a young preacher in a tale of ambition and corruption.
#         """
#     }
# ]

# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "What do you know about {movie}?"),
#     ("ai", "{answer}")
# ])

# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     examples=examples,
#     example_prompt=example_prompt,
# )

# movie_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a movie expert"),
#     few_shot_prompt,
#     ("human", "What do you know about {movie}?")
# ])

# movie_chain = movie_prompt | chat

# movie_chain.invoke({"movie": "The Grand Budapest Hotel"})

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: how to make an italian carbonara"
  ]
}
Certainly! Here's a classic Italian Carbonara recipe for you:

**Ingredients:**
- 400g (14 oz) spaghetti or your preferred pasta
- 150g (5 oz) guanciale (or pancetta if guanciale isn't available), diced
- 3 large eggs
- 1 large egg yolk
- 100g (3.5 oz) Pecorino Romano cheese, finely grated
- Freshly ground black pepper
- Salt (for pasta water)

**Instructions:**

1. **Cook the Pasta:**
   - Bring a large pot of salted water to a boil.
   - Add the pasta and cook until al dente according to package instructions.
   - Reserve about 1 cup of pasta cooking water, then drain the pasta.

2. **Prepare the Sauce:**
   - In a bowl, whisk together the eggs, egg yolk, grated Pecorino Romano, and a generous amount of freshly ground black pepper until well combined.

3. **Cook the Guanciale:**
   - While the pasta cooks, heat a large skillet over medium heat.
   - Add the

"Certainly! Here's a classic Italian Carbonara recipe for you:\n\n**Ingredients:**\n- 400g (14 oz) spaghetti or your preferred pasta\n- 150g (5 oz) guanciale (or pancetta if guanciale isn't available), diced\n- 3 large eggs\n- 1 large egg yolk\n- 100g (3.5 oz) Pecorino Romano cheese, finely grated\n- Freshly ground black pepper\n- Salt (for pasta water)\n\n**Instructions:**\n\n1. **Cook the Pasta:**\n   - Bring a large pot of salted water to a boil.\n   - Add the pasta and cook until al dente according to package instructions.\n   - Reserve about 1 cup of pasta cooking water, then drain the pasta.\n\n2. **Prepare the Sauce:**\n   - In a bowl, whisk together the eggs, egg yolk, grated Pecorino Romano, and a generous amount of freshly ground black pepper until well combined.\n\n3. **Cook the Guanciale:**\n   - While the pasta cooks, heat a large skillet over medium heat.\n   - Add the diced guanciale and cook until crispy and golden, about 4-5 minutes.\n   - Remove from heat.\n\n4. **Com